In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
%cd S:\PPMI_RNA\Processed\Progression_Analysis\Male\HYS_1vs2vs3
data = pd.read_csv('CountData_HYS1vs2vs3_PD_withHYS_Male.csv')
data = data[data['HYS'] != 1]
data.head(1)

S:\PPMI_RNA\Processed\Progression_Analysis\Male\HYS_1vs2vs3


,HudAlphaID,HYS,ENSG00000223972.5,ENSG00000227232.5,ENSG00000278267.1,ENSG00000243485.5,ENSG00000284332.1,ENSG00000237613.2,ENSG00000268020.3,ENSG00000240361.2,...,ENSG00000273739.1,ENSG00000276700.1,ENSG00000276312.1,ENSG00000275757.1,ENSG00000278573.1,ENSG00000276017.1,ENSG00000278817.1,ENSG00000277196.4,ENSG00000278625.1,ENSG00000277374.1
0,5104_SL_0001,2,15,232,15,0,0,0,0,0,...,0,1,16,1,0,2,19,0,0,0


In [3]:
X = data.drop(['HudAlphaID', 'HYS'], axis = 1)
y = data['HYS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   ENSG00000223972.5  ENSG00000227232.5  ENSG00000278267.1  ENSG00000243485.5  \
0                 15                232                 15                  0   
1                  0                 97                  8                  0   

   ENSG00000284332.1  ENSG00000237613.2  ENSG00000268020.3  ENSG00000240361.2  \
0                  0                  0                  0                  0   
1                  0                  0                  0                  0   

   ENSG00000186092.6  ENSG00000238009.6  ...  ENSG00000273739.1  \
0                  0                124  ...                  0   
1                  2                124  ...                  0   

   ENSG00000276700.1  ENSG00000276312.1  ENSG00000275757.1  ENSG00000278573.1  \
0                  1                 16                  1                  0   
1                  9                  0                  1                  0   

   ENSG00000276017.1  ENSG00000278817.1  ENSG00000277196.4  ENSG00

In [4]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [5]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.9813084112149533

In [6]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100


BorutaPy finished running.

Iteration: 	9 / 100
Confirmed: 	0
Tentative: 	0
Rejected: 	58780


BorutaPy(estimator=RandomForestClassifier(n_estimators=3428,
                                          random_state=RandomState(MT19937) at 0x1C7A54A1D40),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1C7A54A1D40, verbose=1)

In [7]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  0


In [8]:
selected_rf_features = pd.DataFrame({'GeneID':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [9]:
Top50 = selected_rf_features.iloc[:50, :]
Top50.head(3)

,GeneID,Ranking
23986,ENSG00000129422.14,2
45173,ENSG00000262823.1,2
50391,ENSG00000197134.11,2


### Top 50

In [10]:
Top50.to_csv ('HYS2vs3_Boruta_Top50_Male.csv', index = False)
selected_rf_features.to_csv ('HYS2vs3_Boruta_All_Male.csv', index = False)

In [11]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['HYS'] = data['HYS']
df_subset.to_csv ('HYS2vs3_Boruta_Top50_Data_Male.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_9404\1943946766.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['HYS'] = data['HYS']


,ENSG00000230679.1,ENSG00000229994.1,ENSG00000238287.1,ENSG00000281912.1,ENSG00000081985.11,ENSG00000270339.3,ENSG00000143365.17,ENSG00000227591.5,ENSG00000214711.9,ENSG00000057935.13,...,ENSG00000259843.2,ENSG00000131153.8,ENSG00000262823.1,ENSG00000166582.9,ENSG00000197134.11,ENSG00000273047.1,ENSG00000100359.20,ENSG00000196507.10,ENSG00000215580.10,HYS
0,30,115,121,28,455,151,188,37,112,836,...,18,120,103,262,552,0,117,130,751,2
1,20,90,60,19,294,123,92,24,93,573,...,10,54,66,94,352,0,142,113,404,2


### Top 100

In [12]:
Top100 = selected_rf_features.iloc[:100, :]
Top100.to_csv ('HYS2vs3_Boruta_Top100_Male.csv', index = False)
Top100.head(2)

,GeneID,Ranking
23986,ENSG00000129422.14,2
45173,ENSG00000262823.1,2


In [13]:
# create a list of column names to subset from data
col_names = Top100.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['HYS'] = data['HYS']
df_subset.to_csv ('HYS2vs3_Boruta_Top100_Data_Male.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_9404\2032703868.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['HYS'] = data['HYS']


,ENSG00000271746.1,ENSG00000230679.1,ENSG00000229994.1,ENSG00000238287.1,ENSG00000162415.6,ENSG00000281912.1,ENSG00000116641.17,ENSG00000081985.11,ENSG00000069702.10,ENSG00000162676.11,...,ENSG00000273047.1,ENSG00000236499.2,ENSG00000211660.3,ENSG00000178026.12,ENSG00000100359.20,ENSG00000196507.10,ENSG00000182508.13,ENSG00000215580.10,ENSG00000229236.1,HYS
0,82,30,115,121,212,28,1178,455,3722,910,...,0,70,1209,6,117,130,32,751,295,2
1,34,20,90,60,193,19,930,294,2289,343,...,0,68,48,10,142,113,20,404,330,2


### Top 250

In [14]:
Top250 = selected_rf_features.iloc[:250, :]
Top250.to_csv ('HYS2vs3_Boruta_Top250_Male.csv', index = False)
Top250.head(2)

,GeneID,Ranking
23986,ENSG00000129422.14,2
45173,ENSG00000262823.1,2


In [15]:
# create a list of column names to subset from data
col_names = Top250.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['HYS'] = data['HYS']
df_subset.to_csv ('HYS2vs3_Boruta_Top250_Data_Male.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_9404\3736889896.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['HYS'] = data['HYS']


,ENSG00000271746.1,ENSG00000171735.18,ENSG00000230679.1,ENSG00000228140.1,ENSG00000130768.14,ENSG00000229994.1,ENSG00000187815.9,ENSG00000238287.1,ENSG00000162415.6,ENSG00000281912.1,...,ENSG00000100197.21,ENSG00000184867.13,ENSG00000158301.18,ENSG00000196507.10,ENSG00000199832.1,ENSG00000182508.13,ENSG00000224485.1,ENSG00000215580.10,ENSG00000229236.1,HYS
0,82,998,30,246,34,115,343,121,212,28,...,13,141,153,130,0,32,0,751,295,2
1,34,748,20,342,35,90,273,60,193,19,...,30,121,91,113,0,20,0,404,330,2


### Top 500

In [16]:
Top500 = selected_rf_features.iloc[:500, :]
Top500.to_csv ('HYS2vs3_Boruta_Top500_Male.csv', index = False)
Top500.head(2)

,GeneID,Ranking
23986,ENSG00000129422.14,2
45173,ENSG00000262823.1,2


In [17]:
# create a list of column names to subset from data
col_names = Top500.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['HYS'] = data['HYS']
df_subset.to_csv ('HYS2vs3_Boruta_Top500_Data_Male.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_9404\1079128826.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['HYS'] = data['HYS']


,ENSG00000271746.1,ENSG00000171735.18,ENSG00000230679.1,ENSG00000142583.17,ENSG00000283789.1,ENSG00000228140.1,ENSG00000117318.8,ENSG00000284657.1,ENSG00000260063.1,ENSG00000130768.14,...,ENSG00000231728.3,ENSG00000199832.1,ENSG00000182508.13,ENSG00000009694.13,ENSG00000029993.14,ENSG00000224485.1,ENSG00000215580.10,ENSG00000229236.1,ENSG00000277196.4,HYS
0,82,998,30,130,2,246,257,142,82,34,...,58,0,32,1194,111,0,751,295,0,2
1,34,748,20,58,0,342,299,117,57,35,...,54,0,20,834,45,0,404,330,1,2
